<a href="https://colab.research.google.com/github/etuckerman/surf_NLP/blob/main/KSF_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pandas scikit-learn numpy sentence-transformers tqdm

In [5]:
# Step 1: Import necessary libraries
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sentence_transformers import SentenceTransformer
import pickle
from tqdm import tqdm

In [6]:
from google.colab import files
import os
# Check if 'messages.csv' exists
if os.path.exists('messages.csv'):
    # Load the CSV file if it exists
    df = pd.read_csv('messages.csv')
    print("File 'messages.csv' found and loaded.")
else:
    # If the file does not exist, prompt the user to upload it
    print("File 'messages.csv' not found. Please upload the file.")
    uploaded = files.upload()
    # Load the uploaded file into a DataFrame
    df = pd.read_csv(next(iter(uploaded.keys())))
    print("File 'messages.csv' uploaded and loaded.")

File 'messages.csv' found and loaded.


In [7]:
# # Assuming the CSV file is called 'messages.csv'
# df = pd.read_csv(list(uploaded.keys())[0])

In [8]:
# Step 3: Data Cleaning
# Convert any non-string values in the 'Message' column to an empty string
df['Message'] = df['Message'].astype(str)

# Apply regex to remove encoded emojis
encoded_emoji_pattern = re.compile(r'[^\x00-\x7F]+')
df['Message'] = df['Message'].apply(lambda x: re.sub(encoded_emoji_pattern, '', x))

# Remove messages with no full words or very short messages
short_responses = ['ok', 'lol', 'haha', 'yes', 'no', 'sure', 'right', 'cool', 'nah', 'yep', 'nope', 'yeah', 'k', 'hm']
df = df[~df['Message'].str.lower().isin(short_responses)]
df = df[df['Message'].str.contains(r'\b\w{3,}\b')]

# Remove empty or very short messages after cleaning
df = df[df['Message'].str.strip().str.len() > 2]

# Reset index after cleaning
df = df.reset_index(drop=True)

In [9]:
# Step 4: Initialize model with GPU support if available
model = SentenceTransformer('paraphrase-MiniLM-L6-v2', device='cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
# Function for batch processing of embeddings
def batch_encode(model, texts, batch_size=32):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        embeddings.extend(model.encode(batch))
    return embeddings

In [11]:
# Generate embeddings using batch processing
df['Embedding'] = batch_encode(model, df['Message'].tolist())

In [12]:
# Save embeddings to a file after computation (optional)
with open('embeddings.pkl', 'wb') as f:
    pickle.dump(df['Embedding'].tolist(), f)

# Load embeddings from a file (optional, for future use)
# with open('embeddings.pkl', 'rb') as f:
#     df['Embedding'] = pickle.load(f)

In [13]:
# Step 5: Function to find the most relevant answer using index as recency
def find_answer(query, df, recency_bias=0.01):
    query_embedding = model.encode(query)


    # Initialize tqdm progress bar
    tqdm.pandas(desc="Finding best answer")

    # Calculate cosine similarity between the query and all messages, progress_apply() for tqdm tracking
    df['Similarity'] = df['Embedding'].progress_apply(lambda x: cosine_similarity([query_embedding], [x])[0][0])

    # Apply recency bias based on index
    df['Recency_Score'] = np.exp(-recency_bias * (len(df) - df.index))

    # Combine similarity and recency scores
    df['Score'] = df['Similarity'] * df['Recency_Score']

    # Find the message with the highest score
    best_match = df.loc[df['Score'].idxmax()]
    return best_match['Message']

In [14]:
# Example usage
query = "how do i strafe?"
answer = find_answer(query, df)
print("Best Answer:", answer)

Finding best answer: 100%|██████████| 658280/658280 [05:54<00:00, 1856.31it/s]

Best Answer: Also you kind of look a bit shaky and reactionary when you board a ramp especially on the blind angles. Get really comfortable with all the ramp boards and flicks in saveloc so you know exactly how the flow of the map is and try to not so abruptly stop but kind of consistently strafe when you're in the air, either towards the side of the next ramp or just weave back and forth if it's straigh in front of you


In [15]:
!pip install -q -U langchain langchain_community transformers bitsandbytes accelerate

In [16]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain import HuggingFacePipeline, PromptTemplate, LLMChain


In [17]:
# Set up the model and tokenizer with 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [18]:
model_name = "mistralai/Mistral-7B-v0.1"
model_4bit = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quantization_config,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
# Create the text generation pipeline
pipeline_inst = pipeline(
    "text-generation",
    model=model_4bit,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_length=2500,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [23]:
# Set up LangChain with the new API
llm = HuggingFacePipeline(pipeline=pipeline_inst)

In [42]:
# Define the template for generating refined responses
template = """<s>[INST] context = You are an assistant for the KSF (Kamikaze Strike Force) Counter-Strike surfing community. Your goal is to provide users with accurate and helpful information about surfing in Counter-Strike, including movement techniques, map recommendations, and community events. Here is the detailed context you need to use:

1. **Surf Movement Basics:**
   - **Strafing:** The primary technique for controlling your movement while surfing. It involves pressing keys to move sideways, allowing you to maintain speed and direction.
   - **Bunnyhopping:** A technique used to gain extra speed while in the air by continuously jumping and strafing.
   - **Ramp Usage:** Understanding how to effectively use ramps to maintain or increase speed.

2. **Common Surf Terms:**
   - **Ramp:** Inclined surfaces that players use to gain speed and navigate maps.
   - **Surf Map:** Custom maps designed specifically for practicing and showcasing surf skills. Maps can range from beginner to expert levels.
   - **Flow:** The smoothness of movement on a map. Good flow involves maintaining speed and momentum through ramps and turns.

3. **Advanced Techniques:**
   - **Wall Surfing:** Using vertical surfaces to maintain or increase speed.
   - **Air Control:** Adjusting your direction and speed while airborne to navigate more effectively.
   - **Speedrun:** Completing a surf map as quickly as possible, often with advanced techniques and precise movements.

4. **Community and Events:**
   - **KSF Events:** Regular community events and competitions where players can showcase their skills and compete for rankings.
   - **Map Reviews:** Feedback and ratings on surf maps submitted by the community.
   - **Tutorials and Guides:** Detailed guides and video tutorials to help players improve their surfing skills and understand advanced techniques.

5. **Troubleshooting and Tips:**
   - **Performance Issues:** Common issues with surf maps and how to optimize settings for better performance.
   - **Common Mistakes:** Frequent errors made by surfers and tips for avoiding them.
   - **Map Recommendations:** Suggestions for surf maps based on skill level and personal preference.

6. **User Interaction:**
   - Provide clear, concise answers based on the context provided.
   - Offer additional resources or links when appropriate.
   - Engage in a friendly and supportive manner, encouraging players to ask more questions or seek help if needed.

Use this context to generate refined answers that help users with their questions about Counter-Strike surfing and KSF community topics.

The user will be asking a question, query, and the most relevent discord message from the community channel will be provided, answer.

Original Query: {query}
Original Answer: {answer}
Context: {context}

Based on the context and the original answer, provide a refined answer to the original query. Make sure your response is relevant, precise, and directly addresses the query.

Refined Answer: [/INST] </s>"""


In [43]:
context = "Counter-Strike surf movement gamemode is a popular game mode where players navigate maps by sliding along surfaces. It is NOT surfing on waves in the sea etc"

In [46]:
from langchain.schema.runnable import RunnableSequence  # Import from the correct module

# Define a function to generate refined answers
def generate_refined_answer(query, answer, context):
    # Use the correct input variable names
    prompt = PromptTemplate(template=template, input_variables=["query", "answer", "context"])
    llm_chain = RunnableSequence(
        prompt, llm
    )
    # Pass the raw string variables to the chain, not the encoded tensors
    response = llm_chain.invoke({"query": query, "answer": answer, "context": context})
    return response

In [47]:
refined_answer = generate_refined_answer(query, answer, context)
print("Refined Answer:", refined_answer)

Refined Answer: <s>[INST] context = You are an assistant for the KSF (Kamikaze Strike Force) Counter-Strike surfing community. Your goal is to provide users with accurate and helpful information about surfing in Counter-Strike, including movement techniques, map recommendations, and community events. Here is the detailed context you need to use:

1. **Surf Movement Basics:**
   - **Strafing:** The primary technique for controlling your movement while surfing. It involves pressing keys to move sideways, allowing you to maintain speed and direction.
   - **Bunnyhopping:** A technique used to gain extra speed while in the air by continuously jumping and strafing.
   - **Ramp Usage:** Understanding how to effectively use ramps to maintain or increase speed.

2. **Common Surf Terms:**
   - **Ramp:** Inclined surfaces that players use to gain speed and navigate maps.
   - **Surf Map:** Custom maps designed specifically for practicing and showcasing surf skills. Maps can range from beginner t